In [19]:
%config Completer.use_jedi = False

In [20]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import numpy as np
import re
import string

Save dataset for handcoding

In [21]:
address_clean = pd.read_pickle('/data/datasets/WOS/US/address_clean.p')

In [22]:
#for repeated runs
#address_clean = address_clean.drop(columns=['usnr_rank','usnr_rank_cat'])

In [23]:
carnegie_universities = address_clean[['grid_id','carnegie_id','carnegie_name']].drop_duplicates('carnegie_id')

carnegie_universities = carnegie_universities.merge(address_clean.carnegie_id.value_counts().to_frame(name='n'), how='left',left_on='carnegie_id',right_index=True)

In [24]:
carnegie_universities = carnegie_universities.sort_values('n',ascending=False)

In [25]:
#carnegie_universities.to_excel('../../handcoding/universities_newsreport_for_handcoding.xlsx', index=False)

Load top 100 universities

In [26]:
us_newsreport = pd.read_excel('../../handcoding/USNEWS Ranking + Carnegie .xlsx', names=['usnr_rank','institution_name','carnegie_id'])

In [27]:
#us_newsreport['usnr_rank'] = us_newsreport.usnr_rank.astype(int)
us_newsreport['carnegie_id'] = us_newsreport.carnegie_id.astype(int).astype('str')

In [28]:
address_clean = address_clean.merge(us_newsreport[['usnr_rank','carnegie_id']], on = 'carnegie_id', how='left')

In [29]:
# I assign the ones that are not in the top 100 to 999
address_clean.loc[address_clean.usnr_rank.isnull(),'usnr_rank'] = 999

In [33]:
test =pd.cut(address_clean.usnr_rank,bins=[0,10,100,999], labels=['top_10','top_100','not_top'])

In [40]:
address_clean['usnr_rank_cat'] = pd.cut(address_clean.usnr_rank,bins=[0,10,100,999], labels=['top_10','top_100','not_top'])

In [41]:
address_clean.usnr_rank_cat.value_counts()

top_100    7818345
not_top    5228800
top_10     2977081
Name: usnr_rank_cat, dtype: int64

In [42]:
address_clean.to_pickle('/data/datasets/WOS/US/address_clean.p')

In [43]:
#address_clean = pd.read_pickle('/data/datasets/WOS/US/address_clean.p')

In [44]:
institutions = address_clean[['carnegie_id','carnegie_name','avg_citations','avg_citations_Q','usnr_rank','usnr_rank_cat','selindex']].drop_duplicates()

In [45]:
institutions.to_csv('../../data/institutions_prestige_metrics.csv', index=False)

In [46]:
address_clean[address_clean.usnr_rank_cat.isnull()]

,cluster_id,id_art,grid_id,carnegie_id,carnegie_name,r1,control,obereg,hbcu,tribal,...,Top5CR_Q,Top1CR_Q,npapers,impact,avg_citations,npapers_Q,impact_Q,avg_citations_Q,usnr_rank,usnr_rank_cat


In [47]:
institutions[institutions.usnr_rank_cat.isnull()]

,carnegie_id,carnegie_name,avg_citations,avg_citations_Q,usnr_rank,usnr_rank_cat,selindex
